In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 401 kB 13.8 MB/s 
     |████████████████████████████████| 19.7 MB 82.4 MB/s 
     |████████████████████████████████| 1.6 MB 62.0 MB/s 
     |████████████████████████████████| 101 kB 10.0 MB/s 
     |████████████████████████████████| 46 kB 4.5 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 4.7 MB 63.4 MB/s 
     |████████████████████████████████| 788 kB 72.5 MB/s 
     |████████████████████████████████| 1.2 MB 83.7 MB/s 
     |████████████████████████████████| 981 kB 64.3 MB/s 
     |████████████████████████████████| 200 kB 75.8 MB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
     |████████████████████████████████| 6.6 MB 77.4 MB/s 
     |████████████████████████████████| 596 kB 82.2 MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-py3-none-any.whl size=116702 sha256=092e9ee2977c

In [ ]:
from flair.data import Corpus, Sentence
from flair.datasets import ColumnCorpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
os.listdir('/content/drive/My Drive/fourthbrain/flair')

['train.txt',
 'test.txt',
 'val.txt',
 'valtrim.txt',
 'ner_test_utf8.txt',
 'ner_val_utf8.txt',
 'ner_train_utf8.txt',
 'weights.txt',
 'training.log',
 'best-model.pt',
 'final-model.pt',
 'dev.tsv',
 'loss.tsv',
 '-resume',
 '-resume2',
 'ner_test_utf8_mod.txt']

In [ ]:
# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = '/content/drive/My Drive/fourthbrain/flair'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='ner_train_utf8.txt',
                              test_file='ner_test_utf8_mod.txt',
                              dev_file='ner_val_utf8.txt'
                               )

2022-08-13 20:32:40,609 Reading data from /content/drive/My Drive/fourthbrain/flair
2022-08-13 20:32:40,611 Train: /content/drive/My Drive/fourthbrain/flair/ner_train_utf8.txt
2022-08-13 20:32:40,612 Dev: /content/drive/My Drive/fourthbrain/flair/ner_val_utf8.txt
2022-08-13 20:32:40,614 Test: /content/drive/My Drive/fourthbrain/flair/ner_test_utf8_mod.txt


In [ ]:
corpus.train[1]

Sentence: "Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as """" Bush Number One Terrorist """" and """" Stop the Bombings ." → ["Bush"/per]

In [ ]:
label_type = 'ner'

label_dict = corpus.make_label_dictionary(label_type=label_type)
label_dict.add_unk = True

2022-08-13 20:32:59,179 Computing label dictionary. Progress:


38557it [00:00, 39985.43it/s]

2022-08-13 20:33:00,200 Dictionary created for label 'ner' with 9 values: geo (seen 30258 times), tim (seen 16316 times), org (seen 16145 times), per (seen 13727 times), gpe (seen 12694 times), art (seen 321 times), eve (seen 261 times), nat (seen 155 times)


In [ ]:
label_dict.get_items()

['<unk>', 'geo', 'tim', 'org', 'per', 'gpe', 'art', 'eve', 'nat']

In [ ]:
embedding_types = [ WordEmbeddings('en-crawl'), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]
embeddings = StackedEmbeddings(embeddings=embedding_types)

2022-08-13 20:33:00,349 https://flair.informatik.hu-berlin.de/resources/embeddings/token/en-fasttext-crawl-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmpmbqqxf2c


100%|██████████| 1200000128/1200000128 [00:38<00:00, 31018799.28B/s]

2022-08-13 20:33:39,102 copying /tmp/tmpmbqqxf2c to cache at /root/.flair/embeddings/en-fasttext-crawl-300d-1M.vectors.npy


2022-08-13 20:33:42,457 removing temp file /tmp/tmpmbqqxf2c
2022-08-13 20:33:42,668 https://flair.informatik.hu-berlin.de/resources/embeddings/token/en-fasttext-crawl-300d-1M not found in cache, downloading to /tmp/tmpff0at2p7


100%|██████████| 39323680/39323680 [00:01<00:00, 24613924.06B/s]

...

2022-08-13 20:33:49,063 copying /tmp/tmp7lokfu34 to cache at /root/.flair/embeddings/lm-news-english-forward-1024-v0.2rc.pt
2022-08-13 20:33:49,087 removing temp file /tmp/tmp7lokfu34


2022-08-13 20:33:59,821 https://flair.informatik.hu-berlin.de/resources/embeddings/flair/lm-news-english-backward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmp0g30_ld_


100%|██████████| 19689779/19689779 [00:00<00:00, 28398395.24B/s]

2022-08-13 20:34:00,581 copying /tmp/tmp0g30_ld_ to cache at /root/.flair/embeddings/lm-news-english-backward-1024-v0.2rc.pt


2022-08-13 20:34:00,604 removing temp file /tmp/tmp0g30_ld_


In [ ]:
tagger = SequenceTagger(hidden_size=256, 
                        embeddings=embeddings, 
                        tag_dictionary=label_dict, 
                        tag_type=label_type)

2022-08-13 20:34:00,667 SequenceTagger predicts: Dictionary with 33 tags: O, S-geo, B-geo, E-geo, I-geo, S-tim, B-tim, E-tim, I-tim, S-org, B-org, E-org, I-org, S-per, B-per, E-per, I-per, S-gpe, B-gpe, E-gpe, I-gpe, S-art, B-art, E-art, I-art, S-eve, B-eve, E-eve, I-eve, S-nat, B-nat, E-nat, I-nat


In [ ]:
trainer = ModelTrainer(tagger, corpus)

In [ ]:
trainer.train('/content/drive/My Drive/fourthbrain/flair/ontonotes-fast',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)

2022-08-13 20:34:00,772 ----------------------------------------------------------------------------------------------------
2022-08-13 20:34:00,774 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'en-crawl'
      (embedding): Embedding(1000001, 300)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.25, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 1024)
        (decoder): Linear(in_features=1024, out_features=275, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.25, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 1024)
        (decoder): Linear(in_features=1024, out_features=275, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=2348, out_features=234

100%|██████████| 150/150 [00:26<00:00,  5.61it/s]

2022-08-13 20:37:03,838 Evaluating as a multi-label problem: False


2022-08-13 20:37:03,930 DEV : loss 0.09992415457963943 - f1-score (micro avg)  0.8186
2022-08-13 20:37:04,408 BAD EPOCHS (no improvement): 0
2022-08-13 20:37:04,412 saving best model
2022-08-13 20:37:11,130 ----------------------------------------------------------------------------------------------------
2022-08-13 20:37:22,213 epoch 2 - iter 120/1205 - loss 0.12793167 - samples/sec: 358.74 - lr: 0.100000
2022-08-13 20:37:33,397 epoch 2 - iter 240/1205 - loss 0.12440011 - samples/sec: 343.76 - lr: 0.100000
2022-08-13 20:37:44,381 epoch 2 - iter 360/1205 - loss 0.12523273 - samples/sec: 349.99 - lr: 0.100000
2022-08-13 20:37:55,159 epoch 2 - iter 480/1205 - loss 0.12530991 - samples/sec: 356.67 - lr: 0.100000
2022-08-13 20:38:05,991 epoch 2 - iter 600/1205 - loss 0.12385342 - samples/sec: 354.91 - lr: 0.100000
2022-08-13 20:38:16,817 epoch 2 - iter 720/1205 - loss 0.12246900 - samples/sec: 355.09 - lr: 0.100000
2022-08-13 20:38:27,497 epoch 2 - iter 840/1205 - loss 0.12136236 - sample

100%|██████████| 150/150 [00:18<00:00,  7.92it/s]


...

2022-08-13 22:55:34,514 Evaluating as a multi-label problem: False
2022-08-13 22:55:34,605 DEV : loss 0.05578828603029251 - f1-score (micro avg)  0.8659
2022-08-13 22:55:35,083 Epoch    65: reducing learning rate of group 0 to 9.7656e-05.
2022-08-13 22:55:35,085 BAD EPOCHS (no improvement): 4
2022-08-13 22:55:35,089 ----------------------------------------------------------------------------------------------------
2022-08-13 22:55:35,091 ----------------------------------------------------------------------------------------------------
2022-08-13 22:55:35,092 learning rate too small - quitting training!
2022-08-13 22:55:35,093 ----------------------------------------------------------------------------------------------------
2022-08-13 22:55:39,727 ----------------------------------------------------------------------------------------------------
2022-08-13 22:55:39,729 loading file /content/drive/My Drive/fourthbrain/flair/ontonotes-fast/best-model.pt
2022-08-13 22:55:43,269 Seque

100%|██████████| 145/145 [00:16<00:00,  8.67it/s]

2022-08-13 22:56:00,300 Evaluating as a multi-label problem: False


2022-08-13 22:56:00,391 0.8552	0.8522	0.8537	0.7922
2022-08-13 22:56:00,393 
Results:
- F-score (micro) 0.8537
- F-score (macro) 0.6707
- Accuracy 0.7922

By class:
              precision    recall  f1-score   support

         geo     0.8476    0.9088    0.8771      3562
         tim     0.9028    0.8652    0.8836      1996
         org     0.7802    0.7149    0.7461      2031
         gpe     0.9593    0.9522    0.9558      1611
         per     0.8097    0.8148    0.8122      1582
         art     0.2857    0.0952    0.1429        42
         eve     0.5500    0.4074    0.4681        27
         nat     0.6000    0.4000    0.4800        15

   micro avg     0.8552    0.8522    0.8537     10866
   macro avg     0.7169    0.6448    0.6707     10866
weighted avg     0.8529    0.8522    0.8516     10866

2022-08-13 22:56:00,395 ----------------------------------------------------------------------------------------------------


{'dev_loss_history': [0.09992415457963943,
  0.07995657622814178,
  0.07152664661407471,
  0.06745276600122452,
  0.06392451375722885,
  0.062401678413152695,
  0.06309185922145844,
  0.06122210621833801,
  0.06174707040190697,
  0.06162280589342117,
  0.05881693214178085,
  0.058544404804706573,
  0.05767440423369408,
  0.05720987543463707,
  0.057569511234760284,
  0.057838182896375656,
  0.05757070705294609,
  0.057493824511766434,
  0.05721729248762131,
  0.05667921528220177,
  0.0575009360909462,
  0.05606365576386452,
  0.056868985295295715,
  0.055931542068719864,
  0.05575476959347725,
  0.05589861795306206,
  0.055802635848522186,
  0.055728621780872345,
  0.05567508935928345,
  0.056234367191791534,
  0.056088197976350784,
  0.05580268055200577,
  0.05570878088474274,
  0.05579906702041626,
  0.05542166903614998,
  0.05563342943787575,
  0.05568663775920868,
  0.055623605847358704,
  0.055258192121982574,
  0.05560987815260887,
  0.05560223385691643,
  0.05562462657690048,
  

In [ ]:
# trained_model = SequenceTagger.load('/content/drive/My Drive/fourthbrain/flair/final-model.pt')

In [ ]:
# path = '/content/drive/My Drive/fourthbrain/flair/'
# trainer.resume(trained_model,
              #  base_path=path + '-resume',
              #  max_epochs=25,
              #  )

In [ ]:
trained_model = SequenceTagger.load('/content/drive/My Drive/fourthbrain/flair/-resume/final-model.pt')

2022-08-13 22:56:00,423 loading file /content/drive/My Drive/fourthbrain/flair/-resume/final-model.pt
2022-08-13 22:56:04,015 SequenceTagger predicts: Dictionary with 35 tags: O, S-geo, B-geo, E-geo, I-geo, S-tim, B-tim, E-tim, I-tim, S-org, B-org, E-org, I-org, S-per, B-per, E-per, I-per, S-gpe, B-gpe, E-gpe, I-gpe, S-art, B-art, E-art, I-art, S-eve, B-eve, E-eve, I-eve, S-nat, B-nat, E-nat, I-nat, <START>, <STOP>


In [ ]:
path = '/content/drive/My Drive/fourthbrain/flair/'
trainer.resume(trained_model,
               base_path=path + '-resume2',
               max_epochs=150,
               )

/usr/local/lib/python3.7/dist-packages/flair/trainers/trainer.py:65: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  "There should be no best model saved at epoch 1 except there "


2022-08-13 22:56:06,841 ----------------------------------------------------------------------------------------------------
2022-08-13 22:56:06,843 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4196, out_features=4196, b

100%|██████████| 150/150 [00:33<00:00,  4.46it/s]


2022-08-13 23:00:22,281 Evaluating as a multi-label problem: False
2022-08-13 23:00:22,368 DEV : loss 0.05820433050394058 - f1-score (micro avg)  0.8601
2022-08-13 23:00:23,215 BAD EPOCHS (no improvement): 0
2022-08-13 23:00:23,223 saving best model
2022-08-13 23:00:24,894 ----------------------------------------------------------------------------------------------------
2022-08-13 23:00:37,178 epoch 27 - iter 120/1205 - loss 0.06910356 - samples/sec: 312.96 - lr: 0.100000
2022-08-13 23:00:49,135 epoch 27 - iter 240/1205 - loss 0.06734562 - samples/sec: 321.46 - lr: 0.100000
2022-08-13 23:01:00,964 epoch 27 - iter 360/1205 - loss 0.06707667 - samples/sec: 324.93 - lr: 0.100000
2022-08-13 23:01:13,050 epoch 27 - iter 480/1205 - loss 0.06757922 - samples/sec: 318.03 - lr: 0.100000
2022-08-13 23:01:24,890 epoch 27 - iter 600/1205 - loss 0.06764711 - samples/sec: 324.66 - lr: 0.100000
2022-08-13 23:01:36,872 epoch 27 - iter 720/1205 - loss 0.06800134 - samples/sec: 320.78 - lr: 0.100000
2

100%|██████████| 150/150 [00:20<00:00,  7.30it/s]

...

2022-08-14 00:51:48,456 Evaluating as a multi-label problem: False


2022-08-14 00:51:48,543 DEV : loss 0.05611134693026543 - f1-score (micro avg)  0.8632
2022-08-14 00:51:49,365 Epoch    48: reducing learning rate of group 0 to 9.7656e-05.
2022-08-14 00:51:49,367 BAD EPOCHS (no improvement): 4
2022-08-14 00:51:49,370 ----------------------------------------------------------------------------------------------------
2022-08-14 00:51:49,372 ----------------------------------------------------------------------------------------------------
2022-08-14 00:51:49,373 learning rate too small - quitting training!
2022-08-14 00:51:49,375 ----------------------------------------------------------------------------------------------------
2022-08-14 00:51:53,060 ----------------------------------------------------------------------------------------------------
2022-08-14 00:51:53,063 loading file /content/drive/My Drive/fourthbrain/flair/-resume2/best-model.pt
2022-08-14 00:51:54,284 SequenceTagger predicts: Dictionary with 35 tags: O, S-geo, B-geo, E-geo, I-ge

100%|██████████| 145/145 [00:22<00:00,  6.53it/s]

2022-08-14 00:52:16,860 Evaluating as a multi-label problem: False
2022-08-14 00:52:16,961 0.853	0.8487	0.8509	0.7885
2022-08-14 00:52:16,963 
Results:
- F-score (micro) 0.8509
- F-score (macro) 0.6593
- Accuracy 0.7885

By class:
              precision    recall  f1-score   support

         geo     0.8503    0.9071    0.8778      3562
         tim     0.9020    0.8717    0.8866      1996
         org     0.7721    0.7041    0.7365      2031
         gpe     0.9551    0.9503    0.9527      1611
         per     0.8062    0.8021    0.8042      1582
         art     0.2632    0.1190    0.1639        42
         eve     0.4545    0.3704    0.4082        27
         nat     0.5000    0.4000    0.4444        15

   micro avg     0.8530    0.8487    0.8509     10866
   macro avg     0.6879    0.6406    0.6593     10866
weighted avg     0.8506    0.8487    0.8489     10866



2022-08-14 00:52:16,965 ----------------------------------------------------------------------------------------------------
